### Hamiltonien fermionic

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver # Nous travaillons avec le driver PySCF

# Renseigner la structure de la molécule 
H2_driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735", #Coordonnées xyz H2
    basis='3-21G', #'def2-SVP', # Ensemble de base
)
H2_problem = H2_driver.run() # Electronic Structure Problem


In [2]:
H2_hamiltonian = H2_problem.hamiltonian # Hamiltonien en 1ere quantification
H2_fermionic_op = H2_hamiltonian.second_q_op() # Hamiltonien fermionique


In [4]:
if len(H2_fermionic_op) <= 20:
    print(H2_fermionic_op)
else: # print the first 20 terms of the fermionic Hamiltonian operator of the molecule
    print("\n".join(str(H2_fermionic_op).splitlines()[:22] + ["..."]))

Fermionic Operator
number spin orbitals=8, number terms=528
  -1.2487422554432475 * ( +_0 -_0 )
+ -0.16735956952108966 * ( +_0 -_2 )
+ -0.5567540613297289 * ( +_1 -_1 )
+ -0.20707789056234427 * ( +_1 -_3 )
+ -0.1673595695210895 * ( +_2 -_0 )
+ -0.09070454289658278 * ( +_2 -_2 )
+ -0.20707789056234424 * ( +_3 -_1 )
+ 0.32440168730746466 * ( +_3 -_3 )
+ -1.2487422554432475 * ( +_4 -_4 )
+ -0.16735956952108966 * ( +_4 -_6 )
+ -0.5567540613297289 * ( +_5 -_5 )
+ -0.20707789056234427 * ( +_5 -_7 )
+ -0.1673595695210895 * ( +_6 -_4 )
+ -0.09070454289658278 * ( +_6 -_6 )
+ -0.20707789056234424 * ( +_7 -_5 )
+ 0.32440168730746466 * ( +_7 -_7 )
+ 0.32727785201769993 * ( +_0 +_0 -_0 -_0 )
+ 0.08367974023919306 * ( +_0 +_0 -_2 -_0 )
+ 0.22771178000942122 * ( +_0 +_1 -_1 -_0 )
+ 0.07177423650389209 * ( +_0 +_1 -_3 -_0 )
...


### Construction du Hamiltonien 

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver # Nous travaillons avec le driver PySCF

# Renseigner la structure de la molécule 
H2_driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735", #Coordonnées xyz H2
    basis='3-21G', #'def2-SVP', # Ensemble de base
)
H2_problem = H2_driver.run() # Electronic Structure Problem

In [4]:

import numpy as np
from pyscf import ao2mo
from qiskit_nature.second_q.operators.symmetric_two_body import fold

In [5]:
mf=H2_driver._calc # champs moyen

In [6]:
## integrales a 1 electron

mo_coeff, mo_coeff_b = H2_driver._expand_mo_object(mf.mo_coeff, array_dimension=3)
h1_a = mf.get_hcore()
h1_a = np.dot(np.dot(mo_coeff.T, h1_a), mo_coeff)
if mo_coeff_b is not None:
    h1_b = np.dot(np.dot(mo_coeff_b.T, h1_a), mo_coeff_b)
else:
    h1_b = None

In [7]:
## Intégrale à 2 electrons

h2_aa = H2_driver._mol.intor("int2e", aosym=8)
h2_aa = fold(ao2mo.full(H2_driver._mol, mo_coeff, aosym=4))
if mo_coeff_b is not None:
    h2_bb = fold(ao2mo.full(H2_driver._mol, mo_coeff_b, aosym=4))
    h2_ba = fold(ao2mo.general(
            H2_driver._mol,
            [mo_coeff_b, mo_coeff_b, mo_coeff, mo_coeff],
            aosym=4,
        )
    )
else:
    h2_bb = None
    h2_ba = None

In [8]:
 from qiskit_nature.second_q.hamiltonians import ElectronicEnergy

In [9]:
  
hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa, h1_b, h2_bb, h2_ba)
hamiltonian.nuclear_repulsion_energy = 1.0

In [10]:
from qiskit_nature.second_q.operators import PolynomialTensor

e_nuc = hamiltonian.nuclear_repulsion_energy
hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
hamiltonian.nuclear_repulsion_energy = None

In [11]:
hamiltonian

In [12]:
from qiskit_nature import second_q 
from qiskit_nature.second_q.operators import ElectronicIntegrals, FermionicOp, PolynomialTensor

In [38]:
#FermionicOp.from_polynomial_tensor(H2_driver.electronic_integrals.second_q_coeffs())

In [13]:
fermionic_op = hamiltonian.second_q_op() # Hamiltonien fermionique

In [14]:
if len(fermionic_op) <= 20:
    print(fermionic_op)
else: # print the first 20 terms of the fermionic Hamiltonian operator of the molecule
    print("\n".join(str(fermionic_op).splitlines()[:22] + ["..."]))

Fermionic Operator
number spin orbitals=8, number terms=529
  1.0
+ -1.2487422554432475 * ( +_0 -_0 )
+ -0.16735956952109032 * ( +_0 -_2 )
+ -0.5567540613297285 * ( +_1 -_1 )
+ -0.20707789056234327 * ( +_1 -_3 )
+ -0.16735956952109027 * ( +_2 -_0 )
+ -0.090704542896583 * ( +_2 -_2 )
+ -0.20707789056234346 * ( +_3 -_1 )
+ 0.32440168730746477 * ( +_3 -_3 )
+ -1.2487422554432475 * ( +_4 -_4 )
+ -0.16735956952109032 * ( +_4 -_6 )
+ -0.5567540613297285 * ( +_5 -_5 )
+ -0.20707789056234327 * ( +_5 -_7 )
+ -0.16735956952109027 * ( +_6 -_4 )
+ -0.090704542896583 * ( +_6 -_6 )
+ -0.20707789056234346 * ( +_7 -_5 )
+ 0.32440168730746477 * ( +_7 -_7 )
+ 0.32727785201769993 * ( +_0 +_0 -_0 -_0 )
+ 0.0836797402391932 * ( +_0 +_0 -_2 -_0 )
+ 0.2277117800094211 * ( +_0 +_1 -_1 -_0 )
...


Après construction du Hamiltonien on obtiens les memes operateurs fermioniques ,mais avec 529 termes au lieu de 528

Autre facon de construire le Hamiltonien

In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver # Nous travaillons avec le driver PySCF

# Renseigner la structure de la molécule 
H2_driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735", #Coordonnées xyz H2
    basis='3-21G', #'def2-SVP', # Ensemble de base
)
H2_problem = H2_driver.run() # Electronic Structure Problem

In [3]:
from qiskit_nature.second_q.formats.qcschema import QCSchema
from qiskit_nature.second_q.formats import qcschema_translator

qcschema = H2_driver.to_qcschema(include_dipole=True)
hamiltonian33 = qcschema_translator._get_mo_hamiltonian_direct(qcschema)

In [4]:
hamiltonian33

In [5]:
fermionic_op=hamiltonian33.second_q_op()

In [6]:
if len(fermionic_op) <= 20:
   print(fermionic_op)
else:
   print("\n".join(str(fermionic_op).splitlines()[:22]+["..."]))   

Fermionic Operator
number spin orbitals=8, number terms=528
  0.32727785201770015 * ( +_0 +_0 -_0 -_0 )
+ 0.08367974023919308 * ( +_0 +_0 -_2 -_0 )
+ 0.2277117800094214 * ( +_0 +_1 -_1 -_0 )
+ 0.07177423650389209 * ( +_0 +_1 -_3 -_0 )
+ 0.08367974023919308 * ( +_0 +_2 -_0 -_0 )
+ 0.2807371312277761 * ( +_0 +_2 -_2 -_0 )
+ 0.07177423650389209 * ( +_0 +_3 -_1 -_0 )
+ 0.33476307215274814 * ( +_0 +_3 -_3 -_0 )
+ 0.32727785201770015 * ( +_0 +_4 -_4 -_0 )
+ 0.08367974023919308 * ( +_0 +_4 -_6 -_0 )
+ 0.2277117800094214 * ( +_0 +_5 -_5 -_0 )
+ 0.07177423650389209 * ( +_0 +_5 -_7 -_0 )
+ 0.08367974023919308 * ( +_0 +_6 -_4 -_0 )
+ 0.2807371312277761 * ( +_0 +_6 -_6 -_0 )
+ 0.07177423650389209 * ( +_0 +_7 -_5 -_0 )
+ 0.33476307215274814 * ( +_0 +_7 -_7 -_0 )
+ 0.04422158915970546 * ( +_0 +_0 -_1 -_1 )
+ 0.040009488695227295 * ( +_0 +_0 -_3 -_1 )
+ 0.04422158915970546 * ( +_0 +_1 -_0 -_1 )
+ -0.008645079696048438 * ( +_0 +_1 -_2 -_1 )
...


In [ ]:
On a à nouveau 28 termes 